# Hito 9
## Comprimiendo con `zlib`

Para entender mejor cómo es un `chunk`, hemos hecho una ilustración simple:

![estructura-de-un-chunk](https://github.com/RaquelGG/TM/blob/master/otros/transponer1.png?raw=true)
### Método `pack()`

In [1]:
def pack(self, seq, chunk):

SyntaxError: unexpected EOF while parsing (<ipython-input-1-74bd1cb10ec8>, line 1)

Hemos decidido comprobar por nuestra cuenta la diferencia de eficiencia entre separar los canales de cada frame en python, de forma normal o usando `numpy`, para ello usamos `timeit`, que cronometra únicamente la parte `stmt` y en setup creamos un array aleatorio, todos del mismo tamaño.

In [44]:
import timeit
py = timeit.timeit(stmt='test[::2]', setup='import os;test=os.urandom(2**15)')
np_sliced = timeit.timeit(stmt='test[::2]', setup='import os;import numpy;test=numpy.frombuffer(os.urandom(2**15),dtype="int16")')
np_trasposed = timeit.timeit(stmt='test.transpose()', setup='import os;import numpy;test=numpy.frombuffer(os.urandom(2**15),dtype="int16").reshape(-1, 2)')
print("con python:", py, "s")
print("con numpy slice :", np_sliced, "s")
print("con numpy transpose :", np_trasposed, "s")

con python: 8.919459699995059 s
con numpy slice : 0.1722801000069012 s
con numpy transpose : 0.14907310000126017 s


Para mejorar la comprensión del código, trasponemos la matriz `chunk`, pasará a ser:

![estructura-de-un-chunk](https://github.com/RaquelGG/TM/blob/master/otros/transponer2.png?raw=true)



In [13]:
import numpy as np
chunk = np.array([[[1], [1]],
                  [[2], [2]],
                  [[3], [3]]])
chunk

array([[[1],
        [1]],

       [[2],
        [2]],

       [[3],
        [3]]])

In [14]:
chunk.transpose()

array([[[1, 2, 3],
        [1, 2, 3]]])

De esta manera solo recorreríamos los canales y queda más legible a la hora de comprimir con `zlib.compress()`:

In [19]:
import zlib

compressed_channels = [zlib.compress(np.ascontiguousarray(channel), level=zlib.Z_BEST_COMPRESSION) for channel in chunk.transpose()]

Codificamos los datos en una secuencia de bytes que pueden ser enviadas por UDP, añadiendo, adicionalmente, el tamaño del primer canal comprimido (para luego poder diferenciar los canales)

In [ ]:
pack_format = f"HH{len(compressed_channels[0])}s{len(compressed_channels[1])}s"

Y por último, empaquetamos los datos:

In [ ]:
return struct.pack(
    pack_format, 
    seq, 
    len(compressed_channels[0]), # tamaño del primer canal comprimido
    *compressed_channels, # * es para compressed_channel[0], [1], ... (expande el array)
)

### Método `unpack()`

In [ ]:
def unpack(self, packed_chunk):

Obtenemos el tamaño del primer canal y del segundo, que se calcula con la diferencia de `packed_chunk` y el tamaño del primer canal:

In [ ]:
first_channel_size, = struct.unpack("H", packed_chunk[SEQ_NO_SIZE:2*SEQ_NO_SIZE])
second_channel_size = len(packed_chunk) - first_channel_size - 2*SEQ_NO_SIZE

Una vez sabemos el tamaño de ambos canales, podemos desempaquetarlo por completo

In [ ]:
seq, _, first_channel_bytes, second_channel_bytes = struct.unpack(
    f"HH{first_channel_size}s{second_channel_size}s",
    packed_chunk,
)

Descomprimimos los canales

In [ ]:
first_channel = np.frombuffer(
    zlib.decompress(first_channel_bytes), 
    dtype='int16',
)   
second_channel = np.frombuffer(
    zlib.decompress(second_channel_bytes),
    dtype='int16'
)

Y por último, volvemos a dejar el chunk con su forma original:

In [ ]:
np.ascontiguousarray(np.concatenate((first_channel, second_channel)).reshape(2,-1).transpose())

![estructura-de-un-chunk](https://github.com/RaquelGG/TM/blob/master/otros/transponer1.png?raw=true)



## Pruebas
En el método `pack()` hemos calculado la tasa de compresión:

In [ ]:
size = sum(len(channel) for channel in compressed_channels)
print("size", size, "bytes, compression rate", "{:.2f}%".format(100*(1-size/4096)))

In [ ]:
Como resultado:
size 1985 bytes, compression rate 51.54%
size 1982 bytes, compression rate 51.61%
size 2017 bytes, compression rate 50.76%
size 2011 bytes, compression rate 50.90%
size 1987 bytes, compression rate 51.49%
size 2018 bytes, compression rate 50.73%
size 1994 bytes, compression rate 51.32%
size 2065 bytes, compression rate 49.58%
size 2056 bytes, compression rate 49.80%
size 1992 bytes, compression rate 51.37%
size 2034 bytes, compression rate 50.34%
size 1933 bytes, compression rate 52.81%

En este caso, la compresión ha mejorado el tamaño de los paquetes considerablemente.

## Comprimiendo directamente el chunk

In [ ]:
def pack(self, seq, chunk):
    """TODO
        """
    compressed_chunk = zlib.compress(chunk.transpose().reshape(-1)) # reshape(-1) deja en una línea el array
    size = len(compressed_chunk)
    print("size", size, "bytes, compression rate", "{:.2f}%".format(100*(1-size/4096)))


    pack_format = f"H{len(compressed_chunk)}s"
    return struct.pack(
        pack_format, 
        seq, 
        compressed_chunk,
    )

def unpack(self, packed_chunk):
    """TODO
        """
    seq, compressed_chunk_bytes = struct.unpack(f"H {len(packed_chunk) - SEQ_NO_SIZE}s", packed_chunk)
        
    chunk = np.frombuffer(
        zlib.decompress(compressed_chunk_bytes), 
        dtype='int16',
    )

    return seq, np.ascontiguousarray(chunk.reshape(2,-1).transpose())

In [ ]:
size 2112 bytes, compression rate 48.44%
size 2046 bytes, compression rate 50.05%
size 2009 bytes, compression rate 50.95%
size 2128 bytes, compression rate 48.05%
size 2067 bytes, compression rate 49.54%
size 1975 bytes, compression rate 51.78%
size 1997 bytes, compression rate 51.25%
size 1947 bytes, compression rate 52.47%
size 2057 bytes, compression rate 49.78%
size 2025 bytes, compression rate 50.56%
size 2061 bytes, compression rate 49.68%
size 2216 bytes, compression rate 45.90%

# Pero... ¿Cómo se escucha?
Se escucha perfecto, no hay diferencia entre un audio sin comprimir y otro comprimido ya que es una compresión sin pérdidas.

![Happy](https://www.freeiconspng.com/uploads/happy-cat-png-0.png)